In [1]:
from keras.preprocessing.sequence import TimeseriesGenerator
import numpy as np

import warnings
warnings.filterwarnings('ignore')

import torch
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
from torch.utils.data import Dataset

import torch.nn.functional as F
import torch.nn as nn
from torch.autograd import Variable

import torchvision
from torchvision import transforms
from torchsummary import summary

from albumentations import (
    ToFloat, 
    CLAHE, 
    RandomRotate90, 
    Transpose, 
    ShiftScaleRotate, 
    Blur, 
    OpticalDistortion, 
    GridDistortion, 
    HueSaturationValue, 
    IAAAdditiveGaussianNoise, 
    GaussNoise, 
    MotionBlur, 
    MedianBlur, 
    IAAPiecewiseAffine, 
    IAASharpen, 
    IAAEmboss, 
    RandomContrast, 
    RandomBrightness, 
    Flip, 
    OneOf, 
    Compose
)

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pylab import rcParams
import numbers
from sklearn.model_selection import train_test_split

from PIL import Image
import math
from tqdm import tqdm

import os
from tqdm import tqdm_notebook as tqdm

# Ignore  the warnings
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd

import warnings
import matplotlib.pyplot as plt
from matplotlib.pyplot import subplots
import matplotlib.patches as patches
import seaborn as sns
from pylab import rcParams

%matplotlib inline 
plt.style.use('seaborn')
sns.set(style='whitegrid',color_codes=True)

# classifiaction 
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier 
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
import xgboost as xgb
import catboost as ctb

# for classification
from sklearn.metrics import accuracy_score

# model selection
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.model_selection import GridSearchCV

# Hp optimization imports
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
import mlflow

import re
import eli5
import gc
import random    
import math
import psutil
import pickle
import datetime
from time import time

# save/load models
from joblib import dump
from joblib import load


import numpy
from keras.optimizers import Adam
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import RepeatVector
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, LSTM, InputLayer, Bidirectional, TimeDistributed, Embedding, Activation
from keras.optimizers import Adam
# fix random seed for reproducibility
numpy.random.seed(7)
from keras.callbacks.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder
from sklearn.model_selection import StratifiedKFold, KFold, RepeatedKFold, train_test_split, GroupKFold, GroupShuffleSplit

from numpy import zeros, newaxis

Using TensorFlow backend.
/Users/kirillvladimirov/PycharmProjects/cv_projects/venv/lib/python3.7/site-packages/eli5/base_utils.py:28: DeprecationWarning: inspect.getargspec() is deprecated since Python 3.0, use inspect.signature() or inspect.getfullargspec()
  init_args = inspect.getargspec(class_.__init__)
/Users/kirillvladimirov/PycharmProjects/cv_projects/venv/lib/python3.7/site-packages/eli5/base_utils.py:36: DeprecationWarning: The usage of `cmp` is deprecated and will be removed on or after 2021-06-01.  Please use `eq` and `order` instead.
  return attr.s(class_, these=these, init=False, slots=True, **attrs_kwargs)  # type: ignore


In [2]:
from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers.core import Activation, Dropout, Dense
from keras.layers import Flatten, LSTM
from keras.layers import GlobalMaxPooling1D
from keras.models import Model
from keras.layers.embeddings import Embedding
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.layers import Input
from keras.layers.merge import Concatenate
from keras.layers import Bidirectional
from keras.models import load_model

import pandas as pd
import numpy as np
import re

import matplotlib.pyplot as plt

In [3]:
root = "../../data/raw/Gamma_Log_Facies_Type_Prediction/"
models_root = "../../models/Gamma_Log_Facies_Type_Prediction/"
submit_root = "../../data/processed/Gamma_Log_Facies_Type_Prediction/"
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)
pd.set_option('max_columns', 150)
# rcParams['figure.figsize'] = 16,8

In [4]:
exp_count = 3

In [5]:
model_file = models_root + f'k{exp_count}_baseline_best_model.h5'
submit_file = submit_root + f'k{exp_count}_baseline_submission.csv'

In [6]:
def show_history(history, metric='mse'):
    plt.plot(history.history[metric])
    plt.plot(history.history[f'val_{metric}'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()
    # summarize history for loss
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()

In [7]:
def to_categorical(sequences, categories):
    cat_sequences = []
    for s in sequences:
        cats = []
        for item in s:
            cats.append(np.zeros(categories))
            cats[-1][item] = 1.0
        cat_sequences.append(cats)
    return np.array(cat_sequences)

In [8]:
%%time
full_train_df = pd.read_csv(root + "Train_File.csv")
full_test_df = pd.read_csv(root + "Test_File.csv")
submit_df = pd.read_csv(root + "Submission_File.csv")


CPU times: user 2.79 s, sys: 474 ms, total: 3.26 s
Wall time: 3.3 s


In [9]:
scaler = StandardScaler()
full_train_df[["GR"]] = scaler.fit_transform(full_train_df[["GR"]])
full_test_df[["GR"]] = scaler.transform(full_test_df[["GR"]])

In [10]:
full_train_df.shape, full_test_df.shape, submit_df.shape

((4400000, 4), (2200000, 4), (2200000, 2))

In [11]:
train = np.array([x.values[:,2:3].T for group, x in full_train_df.groupby('well_id')], dtype='float32')
target = np.array([x.values[:,3:].T for group, x in full_train_df.groupby('well_id')], dtype='float32')
test = np.array([x.values[:,3:4].T for group, x in full_test_df.groupby('well_id')], dtype='float32')

In [12]:
train.shape, target.shape, test.shape

((4000, 1, 1100), (4000, 1, 1100), (2000, 1, 1100))

In [13]:
train_X = train.reshape(train.shape[0], train.shape[2])
train_y = target.reshape(target.shape[0], target.shape[2])
test_X = test.reshape(test.shape[0], test.shape[2])

In [14]:
train_X.shape, train_y.shape, test_X.shape

((4000, 1100), (4000, 1100), (2000, 1100))

In [15]:
train_y = train_y.astype(int)
train_y_cat = to_categorical(train_y, 5)

In [16]:
train_X = train_X[:, :, newaxis]
test_X = test_X[:, :, newaxis]

In [17]:
train_X.shape, train_y_cat.shape, test_X.shape

((4000, 1100, 1), (4000, 1100, 5), (2000, 1100, 1))

In [18]:
# cv =     
# public = 

model = Sequential()
model.add(Bidirectional(LSTM(512, input_shape=(1100, 1), return_sequences=True)))
model.add(Dropout(0.2))
model.add(TimeDistributed(Dense(5)))
model.add(Activation('softmax'))

learning_rate = 0.001
epochs = 80
decay_rate = learning_rate / epochs
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)
mc = ModelCheckpoint(model_file, monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)
adam = Adam(learning_rate, decay=decay_rate)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

In [ ]:
%%time

history = model.fit(train_X, train_y_cat, batch_size=128, epochs=epochs, validation_split=0.2, callbacks=[es, mc])

Train on 3200 samples, validate on 800 samples
Epoch 1/80
1408/3200 [============>.................] - ETA: 7:44 - loss: 1.0898 - accuracy: 0.6089

In [ ]:
model.summary()

In [ ]:
show_history(history, 'accuracy')

In [ ]:
model_1 = load_model(models_root+'k02_baseline_best_model.h5')

In [ ]:
%%time

y_hat = model_1.predict(test_X)

In [ ]:
predict = np.argmax(y_hat.reshape((-1,5)), axis=1)

In [ ]:
submit_df["label"] = predict
submit_df.to_csv(submit_file, index=False)
submit_df.head(20)

In [ ]:
# Количество классов для предсказания
g = submit_df["label"].value_counts()
g = g.sort_index().T
plt.bar(g.index, g.values)